Installing huggingface library (known as transformers)


In [2]:
!pip install transformers


[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [8]:
import torch
torch.manual_seed(0)

In [2]:
!pip install datasets transformers huggingface_hub
!apt-get install git-lfs

zsh:1: command not found: apt-get


Accuracy *metric*

In [9]:
import numpy as np
from datasets import load_metric
 
def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
  
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   return {"accuracy": accuracy}

Creating your dataset

In [10]:
import pandas as pd
import random
import re
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

# model_name = "roberta-base"
# model_name = "bert-base-uncased"
# model_name = "albert-base-v2"
model_name = "vinai/bertweet-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3).to(device)

def remove_links_mentions(tweet):
    # Preprocessing the tweets by removing links and mentions
    link_re_pattern = "https?:\/\/t.co/[\w]+"
    mention_re_pattern = "@\w+"
    tweet = re.sub(link_re_pattern, "", tweet)
    tweet = re.sub(mention_re_pattern, "", tweet)
    return tweet.lower()

def read_tweets_data(tweet_file):
    df = pd.read_csv(tweet_file, sep=',', header=0)
    dic_result = {}
    df1 = df[['tweet_id', 'text', 'airline_sentiment']]
    count = 0
    for index in range(len(df1)):
        try:
            label = df.loc[index, "airline_sentiment"]
            if label == "postive":
              label = 2
            elif label == "negative":
              label = 0
            else:
              label = 1

            tweet_id = df.loc[index, "tweet_id"]
            if label in dic_result:
                dic_result[label][tweet_id] = remove_links_mentions(df.loc[index, "text"])
            else:
                dic_result[label] = {tweet_id: remove_links_mentions(df.loc[index, "text"])}
        except:
            count += 1
    return dic_result


def split_data(twitter_data):
    training = {}
    validation = {}
    test = {}

    for label in twitter_data:
        temp_dic = twitter_data[label]
        lst_tweet_ids = list(temp_dic.keys())
        train_length = int(len(lst_tweet_ids) * 0.8)
        train_ids = lst_tweet_ids[:train_length]
        remaining = lst_tweet_ids[train_length:]
        test_lenght = int(len(remaining) * 0.5)
        test_ids = remaining[:test_lenght]
        validation_id = remaining[test_lenght:]

        for tweet_id in train_ids:
            training[temp_dic[tweet_id]] = label
        for tweet_id in validation_id:
            validation[temp_dic[tweet_id]] = label
        for tweet_id in test_ids:
            test[temp_dic[tweet_id]] = label

    return training, validation, test

dic_tweets = read_tweets_data("Tweets.csv")
training, validation, test = split_data(dic_tweets)

train_encodings = tokenizer(list(training.keys()), truncation=True, padding=True)
val_encodings = tokenizer(list(validation.keys()), truncation=True, padding=True)
test_encodings = tokenizer(list(test.keys()), truncation=True, padding=True)

import torch

class TwitterDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = TwitterDataset(train_encodings, list(training.values()))
val_dataset = TwitterDataset(val_encodings, list(validation.values()))
test_dataset = TwitterDataset(test_encodings, list(test.values()))

cpu


Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /Users/owenbean/.cache/huggingface/hub/models--vinai--bertweet-large/snapshots/67477168d449ccc8abb725e2123a0d6e44f27f4b/config.json
Model config RobertaConfig {
  "_name_or_path": "vinai/bertweet-large",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loadin

In [11]:
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
import numpy as np
from datasets import load_metric
 
def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
  
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   return {"accuracy": accuracy}

model_name = "fine_tuned_twitter_model"
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
training_args = TrainingArguments(
   output_dir=model_name,
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=1,
   save_strategy = "no"#save_strategy="epoch"
)

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=train_dataset,
   eval_dataset=val_dataset,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)


 
trainer.train()
trainer.save_model()
print(trainer.evaluate())
# Test dataset
trainer.eval_dataset = test_dataset
print(trainer.evaluate())
# or
trainer.predict(test_dataset)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/opt/homebrew/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 11375
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 711
  Number of trainable parameters = 355362819


  0%|          | 0/711 [00:00<?, ?it/s]

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 0.3336, 'learning_rate': 5.935302390998594e-06, 'epoch': 0.7}




Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to fine_tuned_twitter_model
Configuration saved in fine_tuned_twitter_model/config.json


...

  0%|          | 0/89 [00:00<?, ?it/s]

***** Running Prediction *****
  Num examples = 1417
  Batch size = 16


{'eval_loss': 0.294400691986084, 'eval_accuracy': 0.89484827099506, 'eval_runtime': 142.7224, 'eval_samples_per_second': 9.928, 'eval_steps_per_second': 0.624, 'epoch': 1.0}


  0%|          | 0/89 [00:00<?, ?it/s]

PredictionOutput(predictions=array([[-0.08183189,  3.8814757 , -4.294788  ],
       [ 1.4584936 ,  2.8027368 , -4.682441  ],
       [ 1.020213  ,  3.0633175 , -4.519721  ],
       ...,
       [ 3.6711974 ,  1.1363608 , -5.2153316 ],
       [ 4.7384696 , -1.1163521 , -4.2427983 ],
       [ 4.729298  , -1.4669893 , -3.9316876 ]], dtype=float32), label_ids=array([1, 1, 1, ..., 0, 0, 0]), metrics={'test_loss': 0.294400691986084, 'test_accuracy': 0.89484827099506, 'test_runtime': 444.348, 'test_samples_per_second': 3.189, 'test_steps_per_second': 0.2})

In case you need to know what libraries are used 

In [ ]:
!pip3 freeze > requirements.txt